In [1]:
BASE_PATH = "/content/drive/MyDrive/Académico/UNIR/MUQC/99 - TFM/dev"

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf

In [4]:
RANDOM_SEED = 1234

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
#tf.random.set_seed(RANDOM_SEED)

In [5]:
# Tamaños finales que queremos para los datasets resultantes
# NOTA: ese 20% de datos de test se verá luego dividido en 10% de validación y 10% de test
# incluyendo qué fracción de datos BENIGN ("1") queremos tener (pues el dataset original no está balanceado)
N_TRAIN = 80000
N_TEST = 20000

N_TRAIN_SMALL = 800
N_TEST_SMALL = 200

N_TRAIN_TINY = 80
N_TEST_TINY = 20

SYN_0_FRACTION = 0.9
SYN_1_FRACTION = 1 - SYN_0_FRACTION

In [6]:
df_tr_day_syn = pd.read_csv(BASE_PATH + "/dataset/CICDDoS2019/CSVs/01-12/Syn.csv")

<ipython-input-6-5eb85b879186>:1: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tr_day_syn = pd.read_csv(BASE_PATH + "/dataset/CICDDoS2019/CSVs/01-12/Syn.csv")


In [7]:
df_tr_day_syn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582681 entries, 0 to 1582680
Data columns (total 88 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Unnamed: 0                    1582681 non-null  int64  
 1   Flow ID                       1582681 non-null  object 
 2    Source IP                    1582681 non-null  object 
 3    Source Port                  1582681 non-null  int64  
 4    Destination IP               1582681 non-null  object 
 5    Destination Port             1582681 non-null  int64  
 6    Protocol                     1582681 non-null  int64  
 7    Timestamp                    1582681 non-null  object 
 8    Flow Duration                1582681 non-null  int64  
 9    Total Fwd Packets            1582681 non-null  int64  
 10   Total Backward Packets       1582681 non-null  int64  
 11  Total Length of Fwd Packets   1582681 non-null  float64
 12   Total Length of Bwd Packets

In [8]:
df_tr_day_syn.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,281052,172.16.0.5-192.168.50.1-53058-53058-6,172.16.0.5,53058,192.168.50.1,53058,6,2018-12-01 13:30:30.741451,115799309,19,...,646237.483665,1709809.0,1.0,1.426117e+07,3.220326e+06,21714933.0,11043464.0,0,1,Syn
1,450424,172.16.0.5-192.168.50.1-32237-32237-6,172.16.0.5,32237,192.168.50.1,32237,6,2018-12-01 13:30:30.741452,113973933,16,...,19.595918,49.0,1.0,1.628198e+07,2.573891e+06,20019405.0,11993631.0,0,1,Syn
2,182979,172.16.0.5-192.168.50.1-60495-9840-6,172.16.0.5,60495,192.168.50.1,9840,6,2018-12-01 13:30:30.741501,112,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
3,41540,172.16.0.5-192.168.50.1-59724-59724-6,172.16.0.5,59724,192.168.50.1,59724,6,2018-12-01 13:30:30.741563,105985004,16,...,17.705259,48.0,1.0,1.514071e+07,3.077366e+06,20954123.0,11120336.0,0,1,Syn
4,358711,172.16.0.5-192.168.50.1-60496-32538-6,172.16.0.5,60496,192.168.50.1,32538,6,2018-12-01 13:30:30.741565,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn


In [9]:
df_test_day_syn = pd.read_csv(BASE_PATH + "/dataset/CICDDoS2019/CSVs/03-11/Syn.csv")

<ipython-input-9-918f12cf3f39>:1: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test_day_syn = pd.read_csv(BASE_PATH + "/dataset/CICDDoS2019/CSVs/03-11/Syn.csv")


In [10]:
df_test_day_syn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320541 entries, 0 to 4320540
Data columns (total 88 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   Flow ID                       object 
 2    Source IP                    object 
 3    Source Port                  int64  
 4    Destination IP               object 
 5    Destination Port             int64  
 6    Protocol                     int64  
 7    Timestamp                    object 
 8    Flow Duration                int64  
 9    Total Fwd Packets            int64  
 10   Total Backward Packets       int64  
 11  Total Length of Fwd Packets   float64
 12   Total Length of Bwd Packets  float64
 13   Fwd Packet Length Max        float64
 14   Fwd Packet Length Min        float64
 15   Fwd Packet Length Mean       float64
 16   Fwd Packet Length Std        float64
 17  Bwd Packet Length Max         float64
 18   Bwd Packet Length Min

In [11]:
df_test_day_syn.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,445444,172.16.0.5-192.168.50.4-9429-9429-6,172.16.0.5,9429,192.168.50.4,9429,6,2018-11-03 11:36:28.607338,36063894,7,...,29.444864,52.0,1.0,1.202128e+07,6.253623e+06,18628035.0,6193840.0,0,1,Syn
1,113842,172.16.0.5-192.168.50.4-60224-60224-6,172.16.0.5,60224,192.168.50.4,60224,6,2018-11-03 11:36:28.607339,44851366,8,...,0.000000,1.0,1.0,2.066268e+07,1.169783e+07,28934293.0,12391060.0,0,1,Syn
2,176377,172.16.0.5-192.168.50.4-33827-11746-6,192.168.50.4,11746,172.16.0.5,33827,6,2018-11-03 11:36:28.607388,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,Syn
3,24777,172.16.0.5-192.168.50.4-33828-1431-6,172.16.0.5,33828,192.168.50.4,1431,6,2018-11-03 11:36:28.607391,0,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
4,85100,172.16.0.5-192.168.50.4-5311-5311-6,172.16.0.5,5311,192.168.50.4,5311,6,2018-11-03 11:36:28.607442,35731470,8,...,33.234019,48.0,1.0,1.191047e+07,1.849493e+06,13693985.0,10001398.0,0,1,Syn


In [12]:
columns_to_remove = ["Unnamed: 0", "Flow ID", " Source IP", " Source Port", " Destination IP", " Destination Port", " Timestamp", "SimillarHTTP"]

df_tr_day_syn.drop(inplace = True, columns = columns_to_remove)
df_test_day_syn.drop(inplace = True, columns = columns_to_remove)

In [13]:
df_tr_day_syn.dropna(inplace = True)
df_test_day_syn.dropna(inplace = True)

In [14]:
df_tr_day_syn.drop_duplicates(inplace = True)
df_test_day_syn.drop_duplicates(inplace = True)

In [15]:
df_tr_day_syn.replace(inplace = True, to_replace={" Label": "Syn"}, value=0)
df_tr_day_syn.replace(inplace = True, to_replace={" Label": "BENIGN"}, value=1)

df_test_day_syn.replace(inplace = True, to_replace={" Label": "Syn"}, value=0)
df_test_day_syn.replace(inplace = True, to_replace={" Label": "BENIGN"}, value=1)

In [16]:
df_tr_day_syn = df_tr_day_syn[np.isfinite(df_tr_day_syn).all(1)]
df_test_day_syn = df_test_day_syn[np.isfinite(df_test_day_syn).all(1)]

In [17]:
df_tr_day_syn[" Label"].value_counts()

 Label
0    155507
1       374
Name: count, dtype: int64

In [18]:
df_tr_day_syn[" Label"].value_counts()[0] / len(df_tr_day_syn[" Label"])

0.9976007338931621

In [19]:
df_test_day_syn[" Label"].value_counts()

 Label
0    452948
1     27052
Name: count, dtype: int64

In [20]:
df_test_day_syn[" Label"].value_counts()[0] / len(df_test_day_syn[" Label"])

0.9436416666666667

In [21]:
# Extraemos cada clase por separado para hacer resampling en cada dataset de training y de test,
# reducir el tamaño de los datasets totales y subir la fracción de la clase "1" (BENIGN)

In [22]:
from sklearn.utils import resample

In [23]:
df_tr_day_syn_0 = df_tr_day_syn[df_tr_day_syn[" Label"] == 0]
df_tr_day_syn_1 = df_tr_day_syn[df_tr_day_syn[" Label"] == 1]

df_test_day_syn_0 = df_test_day_syn[df_test_day_syn[" Label"] == 0]
df_test_day_syn_1 = df_test_day_syn[df_test_day_syn[" Label"] == 1]

In [24]:
df_tr_day_syn_0_resampled = resample(df_tr_day_syn_0,
                                     replace = True,
                                     n_samples = round(N_TRAIN * SYN_0_FRACTION),
                                     random_state = RANDOM_SEED)

df_tr_day_syn_1_resampled = resample(df_tr_day_syn_1,
                                     replace = True,
                                     n_samples = round(N_TRAIN * SYN_1_FRACTION),
                                     random_state = RANDOM_SEED)

df_test_day_syn_0_resampled = resample(df_test_day_syn_0,
                                     replace = True,
                                     n_samples = round(N_TEST * SYN_0_FRACTION),
                                     random_state = RANDOM_SEED)

df_test_day_syn_1_resampled = resample(df_test_day_syn_1,
                                     replace = True,
                                     n_samples = round(N_TEST * SYN_1_FRACTION),
                                     random_state = RANDOM_SEED)

df_tr_day_syn_0_resampled_small = resample(df_tr_day_syn_0,
                                     replace = True,
                                     n_samples = round(N_TRAIN_SMALL * SYN_0_FRACTION),
                                     random_state = RANDOM_SEED)

df_tr_day_syn_1_resampled_small = resample(df_tr_day_syn_1,
                                     replace = True,
                                     n_samples = round(N_TRAIN_SMALL * SYN_1_FRACTION),
                                     random_state = RANDOM_SEED)

df_test_day_syn_0_resampled_small = resample(df_test_day_syn_0,
                                     replace = True,
                                     n_samples = round(N_TEST_SMALL * SYN_0_FRACTION),
                                     random_state = RANDOM_SEED)

df_test_day_syn_1_resampled_small = resample(df_test_day_syn_1,
                                     replace = True,
                                     n_samples = round(N_TEST_SMALL * SYN_1_FRACTION),
                                     random_state = RANDOM_SEED)

df_tr_day_syn_0_resampled_tiny = resample(df_tr_day_syn_0,
                                     replace = True,
                                     n_samples = round(N_TRAIN_TINY * SYN_0_FRACTION),
                                     random_state = RANDOM_SEED)

df_tr_day_syn_1_resampled_tiny = resample(df_tr_day_syn_1,
                                     replace = True,
                                     n_samples = round(N_TRAIN_TINY * SYN_1_FRACTION),
                                     random_state = RANDOM_SEED)

df_test_day_syn_0_resampled_tiny = resample(df_test_day_syn_0,
                                     replace = True,
                                     n_samples = round(N_TEST_TINY * SYN_0_FRACTION),
                                     random_state = RANDOM_SEED)

df_test_day_syn_1_resampled_tiny = resample(df_test_day_syn_1,
                                     replace = True,
                                     n_samples = round(N_TEST_TINY * SYN_1_FRACTION),
                                     random_state = RANDOM_SEED)

In [25]:
# Recombinamos ahora los datasets

In [26]:
df_tr_day_syn_resampled = pd.concat([df_tr_day_syn_0_resampled, df_tr_day_syn_1_resampled])
df_test_day_syn_resampled = pd.concat([df_test_day_syn_0_resampled, df_test_day_syn_1_resampled])

df_tr_day_syn_resampled_small = pd.concat([df_tr_day_syn_0_resampled_small, df_tr_day_syn_1_resampled_small])
df_test_day_syn_resampled_small = pd.concat([df_test_day_syn_0_resampled_small, df_test_day_syn_1_resampled_small])

df_tr_day_syn_resampled_tiny = pd.concat([df_tr_day_syn_0_resampled_tiny, df_tr_day_syn_1_resampled_tiny])
df_test_day_syn_resampled_tiny = pd.concat([df_test_day_syn_0_resampled_tiny, df_test_day_syn_1_resampled_tiny])

In [27]:
df_tr_day_syn_resampled[" Label"].value_counts()

 Label
0    72000
1     8000
Name: count, dtype: int64

In [28]:
df_test_day_syn_resampled[" Label"].value_counts()

 Label
0    18000
1     2000
Name: count, dtype: int64

In [29]:
df_tr_day_syn_resampled_small[" Label"].value_counts()

 Label
0    720
1     80
Name: count, dtype: int64

In [30]:
df_test_day_syn_resampled_small[" Label"].value_counts()

 Label
0    180
1     20
Name: count, dtype: int64

In [31]:
df_tr_day_syn_resampled_tiny[" Label"].value_counts()

 Label
0    72
1     8
Name: count, dtype: int64

In [32]:
df_test_day_syn_resampled_tiny[" Label"].value_counts()

 Label
0    18
1     2
Name: count, dtype: int64

In [33]:
# Ahora guardamos los datasets de nuevo para su uso por el resto de técnicas
# NOTA: Es importante tener en cuenta que cuando los carguemos posteriormente
# lo haremos de forma estratificada en el caso de que usemos funciones como
# train_test_split de sklearn

In [34]:
df_tr_day_syn_resampled.to_csv(BASE_PATH + "/dataset/tr_day_syn.csv", index = False)

In [35]:
df_test_day_syn_resampled.to_csv(BASE_PATH + "/dataset/test_day_syn.csv", index = False)

In [36]:
df_tr_day_syn_resampled_small.to_csv(BASE_PATH + "/dataset/tr_day_syn_small.csv", index = False)

In [37]:
df_test_day_syn_resampled_small.to_csv(BASE_PATH + "/dataset/test_day_syn_small.csv", index = False)

In [38]:
df_tr_day_syn_resampled_tiny.to_csv(BASE_PATH + "/dataset/tr_day_syn_tiny.csv", index = False)

In [39]:
df_test_day_syn_resampled_tiny.to_csv(BASE_PATH + "/dataset/test_day_syn_tiny.csv", index = False)